## Using output_pydantic


The output_pydantic property allows you to define a Pydantic model that the task output should conform to. This ensures that the output is not only structured but also validated according to the Pydantic model.
Here’s an example demonstrating how to use output_pydantic:

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import Agent, Task, Crew,LLM
from pydantic import BaseModel, Field
from typing import List
import json


In [3]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.2,
    api_key=os.getenv("GROQ_API_KEY")
    )

In [5]:
class Blog(BaseModel): # pydantic class 
    title: str
    content: str

In [ ]:
blog_agent = Agent(
    role="Blog Content Generator Agent",
    goal="Generate a blog title and content",
    backstory="""You are an expert content creator, skilled in crafting engaging and informative blog posts.""",
    verbose=True,
    max_tokens = 200,
    llm=llm,
)

task1 = Task(
    description="""
    Topic: {topic}
    Create a blog title and content on a given topic. Make sure the content is under 200 words.""",
    expected_output="A compelling blog title and well-written content.",
    agent=blog_agent,
    output_pydantic = Blog,
)


crew = Crew(
    agents=[blog_agent],
    tasks=[task1],
    verbose=True
)

result = crew.kickoff(inputs={"topic": "The Future of Artificial Intelligence"})


┌───────────────────────── 🚀 Crew Execution Started ─────────────────────────┐
│                                                                             │
│  Crew Execution Started                                                     │
│  Name:                                                                      │
│  crew                                                                       │
│  ID:                                                                        │
│  8f57ed9c-5702-4f84-af46-62b6a7be82e6                                       │
│                                                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

┌────────────────────────────── 📋 Task Started ──────────────────────────────┐
│                                                                             │
│  Task Started                          

┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                   

In [9]:
result

CrewOutput(raw='{\n  "title": "Revolutionizing the World: The Future of Artificial Intelligence",\n  "content": "The future of artificial intelligence (AI) is a topic of much debate and excitement. As AI technology continues to advance at a rapid pace, it is likely to have a significant impact on many aspects of our lives. From virtual assistants to self-driving cars, AI is already being used in a variety of ways. In the future, we can expect to see even more innovative applications of AI, such as personalized medicine and smart homes. However, there are also concerns about the potential risks and challenges associated with AI, such as job displacement and bias in decision-making. As AI continues to evolve, it is essential to consider these issues and work towards creating a future where AI benefits society as a whole."\n}', pydantic=Blog(title='Revolutionizing the World: The Future of Artificial Intelligence', content='The future of artificial intelligence (AI) is a topic of much deba

In [10]:
# Option 1: Accessing Properties Using Dictionary-Style Indexing
print("Accessing Properties - Option 1")
title = result["title"]
content = result["content"]
print("Title:", title)
print("Content:", content)

Accessing Properties - Option 1
Title: Revolutionizing the World: The Future of Artificial Intelligence
Content: The future of artificial intelligence (AI) is a topic of much debate and excitement. As AI technology continues to advance at a rapid pace, it is likely to have a significant impact on many aspects of our lives. From virtual assistants to self-driving cars, AI is already being used in a variety of ways. In the future, we can expect to see even more innovative applications of AI, such as personalized medicine and smart homes. However, there are also concerns about the potential risks and challenges associated with AI, such as job displacement and bias in decision-making. As AI continues to evolve, it is essential to consider these issues and work towards creating a future where AI benefits society as a whole.


In [25]:
# Option 2: Accessing Properties Directly from the Pydantic Model
print("Accessing Properties - Option 2")
title = result.pydantic.title
content = result.pydantic.content
print("Title:", title)
print("Content:", content)


Accessing Properties - Option 2


AttributeError: 'FeedbackSummary' object has no attribute 'title'

In [12]:
# Option 3: Accessing Properties Using the to_dict() Method
print("Accessing Properties - Option 3")
output_dict = result.to_dict()
title = output_dict["title"]
content = output_dict["content"]
print("Title:", title)
print("Content:", content)

Accessing Properties - Option 3
Title: Revolutionizing the World: The Future of Artificial Intelligence
Content: The future of artificial intelligence (AI) is a topic of much debate and excitement. As AI technology continues to advance at a rapid pace, it is likely to have a significant impact on many aspects of our lives. From virtual assistants to self-driving cars, AI is already being used in a variety of ways. In the future, we can expect to see even more innovative applications of AI, such as personalized medicine and smart homes. However, there are also concerns about the potential risks and challenges associated with AI, such as job displacement and bias in decision-making. As AI continues to evolve, it is essential to consider these issues and work towards creating a future where AI benefits society as a whole.


In [13]:

# Option 4: Printing the Entire Blog Object
print("Accessing Properties - Option 5")
print("Blog:", result)

Accessing Properties - Option 5
Blog: title='Revolutionizing the World: The Future of Artificial Intelligence' content='The future of artificial intelligence (AI) is a topic of much debate and excitement. As AI technology continues to advance at a rapid pace, it is likely to have a significant impact on many aspects of our lives. From virtual assistants to self-driving cars, AI is already being used in a variety of ways. In the future, we can expect to see even more innovative applications of AI, such as personalized medicine and smart homes. However, there are also concerns about the potential risks and challenges associated with AI, such as job displacement and bias in decision-making. As AI continues to evolve, it is essential to consider these issues and work towards creating a future where AI benefits society as a whole.'


# SECOND Example

In [ ]:

# Pydantic Models
class SentimentAnalysis(BaseModel): # pydantic models
    overall_sentiment: str = Field(..., description="positive, negative, or neutral")
    sentiment_score: float = Field(..., ge=0, le=1, description="0 to 1 confidence score")
    key_emotions: List[str] = Field(..., description="Main emotions detected")

class FeedbackSummary(BaseModel):
    total_reviews: int = Field(..., description="Number of reviews analyzed")
    sentiment: SentimentAnalysis
    common_complaints: List[str] = Field(..., description="Top 3 complaints")
    common_praises: List[str] = Field(..., description="Top 3 praises")
    improvement_suggestions: List[str] = Field(..., description="Actionable improvements")


In [15]:
# Agent
feedback_analyst = Agent(
    role='Customer Feedback Analyst',
    goal='Analyze customer feedback and provide actionable insights',
    backstory='Expert in sentiment analysis and customer experience',
    llm = llm,
    verbose=True
)

# Task
analysis_task = Task(
    description="""Analyze the following customer reviews: {reviews}
    
    Provide:
    1. Overall sentiment analysis
    2. Common complaints
    3. Common praises
    4. Improvement suggestions
    """,
    agent=feedback_analyst,
    expected_output="Structured feedback analysis with sentiment and insights",
    output_pydantic=FeedbackSummary
)


In [16]:
# Run
crew = Crew(agents=[feedback_analyst], tasks=[analysis_task],verbose=True)

result = crew.kickoff(inputs={
    'reviews': """
    Review 1: Great product but shipping was slow
    Review 2: Love the quality, customer service was excellent
    Review 3: Product arrived damaged, disappointed
    Review 4: Best purchase ever, highly recommend
    Review 5: Good value but instructions unclear
    """
})


┌───────────────────────── 🚀 Crew Execution Started ─────────────────────────┐
│                                                                             │
│  Crew Execution Started                                                     │
│  Name:                                                                      │
│  crew                                                                       │
│  ID:                                                                        │
│  c3994452-1a8d-418f-bbac-631cbb62d973                                       │
│                                                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘



┌────────────────────────────── 📋 Task Started ──────────────────────────────┐
│                                                                             │
│  Task Started                                                               │
│  Name: Analyze the following customer reviews:                              │
│      Review 1: Great product but shipping was slow                          │
│      Review 2: Love the quality, customer service was excellent             │
│      Review 3: Product arrived damaged, disappointed                        │
│      Review 4: Best purchase ever, highly recommend                         │
│      Review 5: Good value but instructions unclear                          │
│                                                                             │
│                                                                             │
│      Provide:                                                               │
│      1. Overall sentiment analysis     

┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                   

In [18]:
print(result)

total_reviews=5 sentiment=SentimentAnalysis(overall_sentiment='positive', sentiment_score=0.6, key_emotions=['satisfaction', 'disappointment', 'excitement']) common_complaints=['slow shipping', 'damaged product', 'unclear instructions'] common_praises=['great product quality', 'excellent customer service', 'good value'] improvement_suggestions=['improve shipping speed', 'enhance product packaging to prevent damage', 'provide clearer instructions for product use']


In [19]:
# Access structured output
summary: FeedbackSummary = analysis_task.output.pydantic
print(f"Sentiment: {summary.sentiment.overall_sentiment}")
print(f"Score: {summary.sentiment.sentiment_score}")
print(f"Complaints: {summary.common_complaints}")
print(f"Praises: {summary.common_praises}")

Sentiment: positive
Score: 0.6
Complaints: ['slow shipping', 'damaged product', 'unclear instructions']
Praises: ['great product quality', 'excellent customer service', 'good value']


In [23]:
# Option 1: Accessing Properties Using Dictionary-Style Indexing
print("Accessing Properties - Option 1")
total_reviews = result["total_reviews"]
key_emotions = result["sentiment"]
common_complaints = result["common_complaints"]
common_praises = result["common_praises"]
improvement_suggestions = result['improvement_suggestions']



Accessing Properties - Option 1


In [24]:
print("total_reviews:", total_reviews)
print("key_emotions:", key_emotions)
print("common_complaints:", common_complaints)
print("common_praises:", common_praises)
print("improvement_suggestions:", improvement_suggestions)

total_reviews: 5
key_emotions: overall_sentiment='positive' sentiment_score=0.6 key_emotions=['satisfaction', 'disappointment', 'excitement']
common_complaints: ['slow shipping', 'damaged product', 'unclear instructions']
common_praises: ['great product quality', 'excellent customer service', 'good value']
improvement_suggestions: ['improve shipping speed', 'enhance product packaging to prevent damage', 'provide clearer instructions for product use']
